# CH-OOD Deep Learning Experiments\n\nThis notebook runs the deep OOD detection experiments for the paper:\n**Certified Geometric OOD via Directional Depth and Kubota Projection Sketches**\n\n## Expected Runtime on T4 GPU\n- **Total time**: ~45-60 minutes\n  - Model training: ~20-25 minutes\n  - Feature extraction: ~10-15 minutes\n  - OOD evaluation: ~15-20 minutes\n\n## Experiments\n- **ID Dataset**: CIFAR-10 (test set)\n- **OOD Datasets**: SVHN, CIFAR-100\n- **Methods**: CH-OOD (ours), Energy, ODIN, Mahalanobis\n- **Metrics**: AUROC, FPR@95%TPR

## Setup Environment

In [ ]:
# Check GPU availability
import torch
if torch.cuda.is_available():
    device = torch.device('cuda')
    print(f"✓ GPU available: {torch.cuda.get_device_name(0)}")
    print(f"  Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
else:
    print("⚠️ No GPU detected. Runtime will be slower.")
    print("To enable GPU in Colab: Runtime → Change runtime type → Hardware accelerator → GPU (T4)")

## Install Dependencies

In [ ]:
# Install required packages (if not in Colab, these should already be available)
!pip install -q torch torchvision numpy scikit-learn matplotlib

## Load Experiment Code

In [ ]:
# Download the experiment code
!wget -q https://raw.githubusercontent.com/ashachar/ch-ood-experiments/main/ch_ood_colab.py
print("✓ Experiment code downloaded")

In [ ]:
# Import the experiment module
import ch_ood_colab
import importlib
importlib.reload(ch_ood_colab)  # Reload in case of updates

## Run Experiments

In [ ]:
# Setup directories and device
device = ch_ood_colab.setup_colab()

### Option 1: Run Complete Pipeline (Recommended)
This will train the model and run all OOD experiments.

In [ ]:
# Run full experiments (train model + evaluate OOD)
# Expected time: ~45-60 minutes on T4 GPU
results = ch_ood_colab.run_deep_ood_experiments(device, train_model=True)

### Option 2: Quick Test (Skip Training)
Use a pre-trained model if available.

In [ ]:
# Quick test with pre-trained model (if available)
# results = ch_ood_colab.run_deep_ood_experiments(device, train_model=False)

## View Results

In [ ]:
# Display summary table
import json
import pandas as pd

# Load results
with open('results/deep_ood_results.json', 'r') as f:
    results = json.load(f)

# Create summary DataFrame
summary_data = []
for ood_name, methods in results.items():
    for method_name, metrics in methods.items():
        summary_data.append({
            'OOD Dataset': ood_name,
            'Method': method_name,
            'AUROC': f"{metrics['auroc']:.3f}",
            'FPR@95%': f"{metrics['fpr95']:.3f}"
        })

df = pd.DataFrame(summary_data)
pivot_auroc = df.pivot(index='Method', columns='OOD Dataset', values='AUROC')
pivot_fpr = df.pivot(index='Method', columns='OOD Dataset', values='FPR@95%')

print("\n" + "="*50)
print("AUROC Results")
print("="*50)
print(pivot_auroc)

print("\n" + "="*50)
print("FPR@95%TPR Results")
print("="*50)
print(pivot_fpr)

## Display ROC Curves

In [ ]:
# Display generated ROC curves
from IPython.display import Image, display
import os

# List all generated figures
figures = [f for f in os.listdir('figures') if f.endswith('.png')]

for fig in figures:
    print(f"\n{fig}:")
    display(Image(f'figures/{fig}'))

## Download Results

To download the results and figures:

In [ ]:
# Create a zip file with all results
import zipfile
import os

with zipfile.ZipFile('ch_ood_results.zip', 'w') as zipf:
    # Add results JSON
    zipf.write('results/deep_ood_results.json')
    
    # Add all figures
    for fig in os.listdir('figures'):
        if fig.endswith('.png'):
            zipf.write(f'figures/{fig}')
    
    # Add trained model if it exists
    if os.path.exists('models/cifar10_resnet18_best.pth'):
        zipf.write('models/cifar10_resnet18_best.pth')

print("✓ Results packaged in ch_ood_results.zip")
print("Download this file to save your results.")

## Ablation Studies

In [ ]:
# Optional: Run ablation studies on hyperparameters
# This section can be used to test different values of alpha, m, etc.

# Example: Test different alpha values for CH-OOD
import numpy as np
import matplotlib.pyplot as plt

# Load features (assuming they're already extracted)
# alphas = [0.03, 0.05, 0.08, 0.10, 0.12, 0.15]
# results_by_alpha = []

# for alpha in alphas:
#     ch_ood = ch_ood_colab.HadwigerSoft.fit(features_id_cal, target_fpr=0.05, alpha=alpha)
#     scores_id = ch_ood.scores(features_id_test)[2]
#     scores_ood = ch_ood.scores(features_ood)[2]
#     auroc, _, _, _ = ch_ood_colab.compute_metrics(scores_ood, scores_id)
#     results_by_alpha.append(auroc)

# plt.figure(figsize=(8, 5))
# plt.plot(alphas, results_by_alpha, 'o-')
# plt.xlabel('Alpha')
# plt.ylabel('AUROC')
# plt.title('AUROC vs Alpha Parameter')
# plt.grid(True, alpha=0.3)
# plt.show()

## Citation

If you use this code in your research, please cite:

```bibtex
@article{shachar2025chood,
  title={Certified Geometric OOD via Directional Depth and Kubota Projection Sketches},
  author={Shachar, Amir},
  year={2025}
}
```